In [1]:
import pandas as pd
import plotly.express as px

In [2]:
df_train = pd.read_csv('EnefitAPI/train.csv')
df_train['datetime'] = df_train['datetime'].astype('datetime64[ns]')
df_weather_station_to_county_mapping = pd.read_csv('EnefitAPI/weather_station_to_county_mapping.csv')
df_gas_prices = pd.read_csv('EnefitAPI/gas_prices.csv')
df_client = pd.read_csv('EnefitAPI/client.csv')
df_elec_prices = pd.read_csv('EnefitAPI/electricity_prices.csv')
df_forecast_weather = pd.read_csv('../../EnefitBigData/forecast_weather.csv')
df_historical_weather = pd.read_csv('../../EnefitBigData/historical_weather.csv')
wscm = pd.read_csv(f'EnefitAPI/weather_station_to_county_mapping.csv')


In [4]:
df_weather_station_to_county_mapping['size'] = 5

fig = px.scatter_mapbox(
    df_weather_station_to_county_mapping, 
    lat="latitude", 
    lon="longitude", 
    color="county",
    size='size',
    zoom=6,
    title='Weather Stations Locations',
    width=1000,
    height=600
)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})
fig.show()

In [4]:
print(df_train.isna().sum(), "\n")
missing = df_train[df_train['target'].isna()==True]['datetime']
print("missing values len: %s" %(len(missing)))
print("missing datetime: %s" %(list(missing.unique())))

county                  0
is_business             0
product_type            0
target                528
is_consumption          0
datetime                0
data_block_id           0
row_id                  0
prediction_unit_id      0
dtype: int64 

missing values len: 528
missing datetime: [numpy.datetime64('2021-10-31T03:00:00.000000000'), numpy.datetime64('2022-03-27T03:00:00.000000000'), numpy.datetime64('2022-10-30T03:00:00.000000000'), numpy.datetime64('2023-03-26T03:00:00.000000000')]


In [5]:
train_dropna = df_train.dropna()
con_train = train_dropna[train_dropna['is_consumption']==1]
pro_train = train_dropna[train_dropna['is_consumption']==0]
len(con_train), len(pro_train)

(1008912, 1008912)

In [6]:
df_forecast_weather.head(5)

,latitude,longitude,origin_datetime,hours_ahead,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,data_block_id,forecast_datetime,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
0,57.6,21.7,2021-09-01 00:00:00+00:00,1,15.655786,11.553613,0.904816,0.019714,0.000000,0.905899,-0.411328,-9.106137,1,2021-09-01 01:00:00+00:00,0.0,0.0,0.0,0.0
1,57.6,22.2,2021-09-01 00:00:00+00:00,1,13.003931,10.689844,0.886322,0.004456,0.000000,0.886658,0.206347,-5.355405,1,2021-09-01 01:00:00+00:00,0.0,0.0,0.0,0.0
2,57.6,22.7,2021-09-01 00:00:00+00:00,1,14.206567,11.671777,0.729034,0.005615,0.000000,0.730499,1.451587,-7.417905,1,2021-09-01 01:00:00+00:00,0.0,0.0,0.0,0.0
3,57.6,23.2,2021-09-01 00:00:00+00:00,1,14.844507,12.264917,0.336304,0.074341,0.000626,0.385468,1.090869,-9.163999,1,2021-09-01 01:00:00+00:00,0.0,0.0,0.0,0.0
4,57.6,23.7,2021-09-01 00:00:00+00:00,1,15.293848,12.458887,0.102875,0.088074,0.000015,0.176590,1.268481,-8.975766,1,2021-09-01 01:00:00+00:00,0.0,0.0,0.0,0.0


In [27]:
df1 = pd.merge(df_train, df_gas_prices[['data_block_id', 'lowest_price_per_mwh', 'highest_price_per_mwh']], on='data_block_id', how='left')
df2 = pd.merge(df1, df_client[['data_block_id', 'product_type', 'county', 'is_business', 'eic_count', 'installed_capacity']], 
           on=['data_block_id', 'product_type', 'county', 'is_business'], how='left')
df_elec_prices['datetime'] = df_elec_prices['forecast_date'].astype('datetime64[ns]') + pd.DateOffset(1)
df3 = pd.merge(df2, df_elec_prices[['data_block_id', 'euros_per_mwh', 'datetime']], on=['datetime', 'data_block_id'], how='left')
wscm['longitude'] = wscm['longitude'].round(1)
wscm['latitude'] = wscm['latitude'].round(1)
df_forecast_weather['datetime'] = df_forecast_weather['forecast_datetime'].astype('datetime64[ns]')
df_forecast_weather = pd.merge(df_forecast_weather, wscm[['latitude', 'longitude', 'county']], on=['latitude', 'longitude'], how='left')
mer_feat = ['temperature', 'dewpoint', 'cloudcover_high', 'cloudcover_low',
       'cloudcover_mid', 'cloudcover_total', '10_metre_u_wind_component',
       '10_metre_v_wind_component',  'direct_solar_radiation', 
            'surface_solar_radiation_downwards', 'snowfall', 
            'total_precipitation', 'datetime', 'county', 'data_block_id']
temp = df_forecast_weather[mer_feat].groupby(['datetime', 'county', 'data_block_id']).mean().reset_index()
df4 = pd.merge(df3, temp, on=['datetime', 'county', 'data_block_id'], how='left')
df4.head(2)

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,lowest_price_per_mwh,...,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
0,0,0,1,0.713,0,2021-09-01,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,1,96.590,1,2021-09-01,0,1,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df_historical_weather['datetime'] = df_historical_weather['datetime'].astype('datetime64[ns]')
df_historical_weather['datetime'] = df_historical_weather['datetime'].apply(lambda x: 
                                                      x + pd.DateOffset(1) if x.hour < 11 else x + pd.DateOffset(2))
df_historical_weather = pd.merge(df_historical_weather, wscm[['latitude', 'longitude', 'county']], on=['latitude', 'longitude'], how='left')
df_historical_weather.head(2)

,datetime,temperature,dewpoint,rain,snowfall,surface_pressure,cloudcover_total,cloudcover_low,cloudcover_mid,cloudcover_high,windspeed_10m,winddirection_10m,shortwave_radiation,direct_solar_radiation,diffuse_radiation,latitude,longitude,data_block_id,county
0,2021-09-02 00:00:00,14.4,12.0,0.0,0.0,1015.8,4,4,0,0,6.694444,3,0.0,0.0,0.0,57.6,21.7,1,NaN
1,2021-09-02 00:00:00,14.0,12.0,0.0,0.0,1010.6,7,8,0,0,4.944444,353,0.0,0.0,0.0,57.6,22.2,1,NaN
2,2021-09-02 00:00:00,14.4,12.8,0.0,0.0,1014.9,6,7,0,0,5.833333,348,0.0,0.0,0.0,57.6,22.7,1,NaN
3,2021-09-02 00:00:00,15.4,13.0,0.0,0.0,1014.4,4,2,4,0,7.111111,349,0.0,0.0,0.0,57.6,23.2,1,NaN
4,2021-09-02 00:00:00,15.9,12.6,0.0,0.0,1013.8,12,7,0,20,8.388889,360,0.0,0.0,0.0,57.6,23.7,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1710795,2023-05-31 10:00:00,10.8,4.1,0.4,0.0,1018.7,40,0,67,0,1.333333,283,440.0,264.0,176.0,59.7,26.2,637,NaN
1710796,2023-05-31 10:00:00,11.4,2.9,0.3,0.0,1018.8,41,0,69,0,1.527778,302,453.0,283.0,170.0,59.7,26.7,637,NaN
1710797,2023-05-31 10:00:00,9.5,3.2,0.1,0.0,1018.9,47,0,79,0,3.416667,302,477.0,312.0,165.0,59.7,27.2,637,NaN
1710798,2023-05-31 10:00:00,11.0,1.6,0.0,0.0,1018.6,31,0,52,0,3.472222,309,535.0,400.0,135.0,59.7,27.7,637,NaN


In [30]:
df4.head(2)

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,lowest_price_per_mwh,...,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation
0,0,0,1,0.713,0,2021-09-01,0,0,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,1,96.590,1,2021-09-01,0,1,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
feat=['temperature', 'dewpoint', 'rain', 'snowfall',
       'surface_pressure', 'cloudcover_total', 'cloudcover_low',
       'cloudcover_mid', 'cloudcover_high', 'windspeed_10m',
       'winddirection_10m', 'shortwave_radiation', 'direct_solar_radiation',
       'diffuse_radiation']
feat_h = ["%s_h"%i for i in feat]
feat_dict = dict(zip(feat, feat_h))
df_historical_weather = df_historical_weather.rename(columns=feat_dict)
df_historical_weather.head(1)

,datetime,temperature_h,dewpoint_h,rain_h,snowfall_h,surface_pressure_h,cloudcover_total_h,cloudcover_low_h,cloudcover_mid_h,cloudcover_high_h,windspeed_10m_h,winddirection_10m_h,shortwave_radiation_h,direct_solar_radiation_h,diffuse_radiation_h,latitude,longitude,data_block_id,county
0,2021-09-02,14.4,12.0,0.0,0.0,1015.8,4,4,0,0,6.694444,3,0.0,0.0,0.0,57.6,21.7,1,NaN


In [32]:
mer_feat = feat_h + ['datetime', 'county', 'data_block_id']
temp = df_historical_weather[mer_feat].groupby(['datetime', 'county', 'data_block_id']).mean().reset_index()
df5 = pd.merge(df4, temp, on=['datetime', 'county', 'data_block_id'], how='left')

In [34]:
df = df5.copy().drop(['row_id', 'prediction_unit_id'], axis=1)
df.to_csv('../../EnefitBigData/merge.csv')

In [36]:
df.head(5)

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,lowest_price_per_mwh,highest_price_per_mwh,eic_count,...,surface_pressure_h,cloudcover_total_h,cloudcover_low_h,cloudcover_mid_h,cloudcover_high_h,windspeed_10m_h,winddirection_10m_h,shortwave_radiation_h,direct_solar_radiation_h,diffuse_radiation_h
0,0,0,1,0.713,0,2021-09-01,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,1,96.590,1,2021-09-01,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,2,0.000,0,2021-09-01,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,2,17.314,1,2021-09-01,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,3,2.904,0,2021-09-01,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
df['target'].mean()

274.8555600988986